In [1]:
from utils.infer import *
import pandas as pd
from  utils.eval import *
from utils.chat import *
from utils.data import *

%reload_ext autoreload
%autoreload 2

/Users/paul/anaconda3/envs/ClinicalNotes/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Combining inferences

In [2]:
inference_paths = "../data/inference"
output_path = "../data/combined_inferences.jsonl"

combined = combine(inference_paths, output_path)

### Building evaluation inputs

In [3]:
build_evaluation_inputs(output_path, models = ['direct-gpt', 'meditron-7b-summarizer', 'meditron-13b-summarizer'])

### Evaluation GPT 3 direct

In [10]:
gpt_3_eval_res = note_evaluation('direct-gpt')

Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'scores': 'tbd', 'bleu': 'done', 'rouge': 'tbd', 'bert': 'done', 'gpt_score': 'done'}
Removing stopwords...
Stopwords removed.
BLEU scores already computed.


Computing Rouge scores: 100%|██████████| 10/10 [05:04<00:00, 30.49s/it]


Computing BERTscores...
BERTscores already computed.
GPT-4 scoring...
GPT-4 scores already computed.


In [11]:
display(gpt_3_eval_res)

,idx,gold,pred,model_name,bleu,rouge1,rouge2,rougeL,rougeLsum,bert,gpt_score
0,102675,A 12-day-old baby boy of Indian origin with a ...,Patient presented with a 12-day-old baby boy o...,direct-gpt,0.193591,0.867925,0.696682,0.528302,0.528302,0.875239,9
1,40735,A 69-year-old man was referred from an outside...,Patient presented to the clinic with a history...,direct-gpt,0.384111,0.536723,0.295455,0.451977,0.451977,0.855199,8
2,195257,The patient is a 47-year-old Caucasian female ...,Patient presented with complaints of left-side...,direct-gpt,0.092334,0.493671,0.280255,0.398734,0.398734,0.836584,7
3,9632,An 8 days old male neonate was born to an Asia...,Patient presented with an 8-day-old baby boy w...,direct-gpt,0.358175,0.719388,0.538462,0.642857,0.642857,0.898979,9
4,82889,A right-handed 80-year-old Caucasian female wa...,Patient presented with a chief complaint of re...,direct-gpt,0.240081,0.685851,0.525301,0.637890,0.637890,0.872304,9
...,...,...,...,...,...,...,...,...,...,...,...
995,9782,The patient was a 9-year-old boy with a histor...,Patient presented with concerns about the loss...,direct-gpt,0.318324,0.481481,0.186567,0.377778,0.377778,0.867252,7
996,20482,The patient is a 72-year-old immunocompetent H...,Patient presented with a chief complaint of a ...,direct-gpt,0.227259,0.417445,0.263323,0.367601,0.367601,0.878446,9
997,48902,"A 28-year-old woman, gravida 3, para 1, was di...",Patient presented with a history of missed mis...,direct-gpt,0.351929,0.542056,0.357367,0.485981,0.485981,0.872541,8
998,5358,An 18 year old woman was referred to our outpa...,Patient presents with a history of hypothyroid...,direct-gpt,0.241049,0.652819,0.429851,0.599407,0.599407,0.838174,9


In [12]:
print(gpt_3_eval_res[['bleu','bert','gpt_score'] + ROUGE_SUB_SCORES].mean())

bleu         0.250703
bert         0.876228
gpt_score    8.196000
rouge1       0.611542
rouge2       0.408162
rougeL       0.531128
rougeLsum    0.531128
dtype: float64


### Evaluation meditron-7b-summarizer

In [13]:
summarizer7_eval_by_sample , summarizer7_eval_by_key = summary_evaluation('meditron-7b-summarizer')

Running summary evaluation with score types: ['bleu', 'rouge', 'bert', 'gpt_score']
Loading data...
Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'flatten and match dicts': 'done', 'clean dicts and counts': 'done', 'summary_statistics': 'in progress', 'scores': 'in progress', 'pairs_idx': 'done', 'eval_by_sample': 'tbd', 'eval_by_key': 'tbd', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}
BLEU scores already computed.
ROUGE scores already computed.
Computing BERTscores...
BERTscores already computed.
GPT-4 scoring...
GPT-4 scores already computed.


In [14]:
display(summarizer7_eval_by_sample)
display(summarizer7_eval_by_key)

,idx,gold,pred,mean_bleu,mean_bert,mean_gpt_score,mean_rouge1,mean_rouge2,mean_rougeL,mean_rougeLsum,missing_keys_count,extra_keys_count,common_none_count,gold_none_count,pred_none_count,common_non_none,all_keys_count
0,127042,{'visit motivation': 'Severe pain and restrict...,{'visit motivation': 'Pain in the right should...,0.024697,0.781193,7.000000,0.300449,0.091925,0.282489,0.282489,7,0,30,3,2,23,65
1,131562,{'visit motivation': 'Abdominal mass that had ...,"{'visit motivation': 'Abdominal mass, loss of ...",0.154609,0.869353,6.888889,0.553576,0.334366,0.537042,0.537042,3,0,38,0,2,18,61
2,141471,{'visit motivation': 'History of flank pain on...,{'visit motivation': 'Right-sided flank pain a...,0.057203,0.835415,5.600000,0.383232,0.206255,0.374036,0.374036,28,0,16,4,4,15,67
3,117562,{'visit motivation': 'Subacute progressive bil...,{'visit motivation': 'Progressive bilateral le...,0.226453,0.888196,7.468750,0.547230,0.420537,0.528320,0.528320,6,0,29,3,5,32,75
4,55332,{'visit motivation': 'Dyspnea since three mont...,{'visit motivation': 'Persistent dyspnea for t...,0.136109,0.893432,7.600000,0.510297,0.347374,0.510297,0.510297,34,0,28,1,3,25,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,93078,{'visit motivation': 'Anterior midline neck ma...,{'visit motivation': 'Anterior midline neck ma...,0.193582,0.898657,7.450000,0.657176,0.503265,0.638362,0.638362,25,0,23,3,2,20,73
996,38574,{'visit motivation': 'unilateral left knee pai...,"{'visit motivation': 'Left knee pain', 'admiss...",0.171704,0.901672,6.312500,0.575768,0.451841,0.563573,0.563573,16,0,19,5,6,16,62
997,65166,"{'visit motivation': 'Painful, swollen right k...",{'visit motivation': 'Painful and swollen righ...,0.210072,0.888840,7.437500,0.618466,0.490061,0.609265,0.609265,0,0,31,2,2,32,67
998,157147,{'visit motivation': 'Acute cerebral air embol...,{'visit motivation': 'Acute onset of unrespons...,0.172424,0.837329,4.722222,0.415853,0.344101,0.415853,0.415853,15,0,34,2,7,18,76


,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,gold_none_prop,pred_none_prop,missing_keys_prop
keys,,,,,,,,,,
admission/0/care center details,0.270086,0.918273,7.611111,0.735385,0.650588,0.726991,0.726991,0.048,0.153,0.012
admission/0/date,0.231097,0.919139,7.827957,0.761162,0.608599,0.754994,0.754994,0.020,0.095,0.012
admission/0/duration,0.116882,0.927714,8.325843,0.762572,0.690028,0.762572,0.762572,0.021,0.049,0.012
admission/0/reason,0.262550,0.884549,7.142216,0.552512,0.429219,0.531419,0.531419,0.038,0.069,0.012
admission/1/care center details,0.082227,0.941237,8.200000,0.795455,0.666667,0.795455,0.795455,0.001,0.003,0.010
...,...,...,...,...,...,...,...,...,...,...
treatments/5/reaction to treatment,0.000000,0.926654,8.000000,0.533333,0.461538,0.533333,0.533333,0.000,0.000,0.016
treatments/5/reason for taking,0.333333,0.809110,4.666667,0.416667,0.333333,0.416667,0.416667,0.000,0.000,0.016
treatments/5/related condition,0.000000,0.833170,5.500000,0.500000,0.000000,0.500000,0.500000,0.000,0.000,0.016


### Evaluation meditron-13b-summarizer

In [16]:
summarizer13_eval_by_sample , summarizer13_eval_by_key = summary_evaluation('meditron-13b-summarizer')

Running summary evaluation with score types: ['bleu', 'rouge', 'bert', 'gpt_score']
Loading data...
Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'flatten and match dicts': 'done', 'clean dicts and counts': 'done', 'summary_statistics': 'in progress', 'scores': 'in progress', 'pairs_idx': 'done', 'eval_by_sample': 'tbd', 'eval_by_key': 'tbd', 'bleu': 'done', 'rouge': 'done', 'bert': 'tbd', 'gpt_score': 'done'}
BLEU scores already computed.
ROUGE scores already computed.
Computing BERTscores...


Computing BERT scores: 100%|██████████| 249/249 [36:28<00:00,  8.79s/it]


GPT-4 scoring...
GPT-4 scores already computed.


In [17]:
display(summarizer13_eval_by_sample)
display(summarizer13_eval_by_key)

,idx,gold,pred,mean_bleu,mean_bert,mean_gpt_score,mean_rouge1,mean_rouge2,mean_rougeL,mean_rougeLsum,missing_keys_count,extra_keys_count,common_none_count,gold_none_count,pred_none_count,common_non_none,all_keys_count
0,127042,{'visit motivation': 'Severe pain and restrict...,{'visit motivation': 'Pain in the right should...,0.024697,0.781193,4.347826,0.300449,0.091925,0.282489,0.282489,7,0,30,3,2,23,65
1,131562,{'visit motivation': 'Abdominal mass that had ...,"{'visit motivation': 'Abdominal mass, loss of ...",0.154609,0.869352,6.555556,0.553576,0.334366,0.537042,0.537042,3,0,38,0,2,18,61
2,141471,{'visit motivation': 'History of flank pain on...,{'visit motivation': 'Right-sided flank pain a...,0.057203,0.835415,5.533333,0.383232,0.206255,0.374036,0.374036,28,0,16,4,4,15,67
3,117562,{'visit motivation': 'Subacute progressive bil...,{'visit motivation': 'Progressive bilateral le...,0.226453,0.888196,7.468750,0.547230,0.420537,0.528320,0.528320,6,0,29,3,5,32,75
4,55332,{'visit motivation': 'Dyspnea since three mont...,{'visit motivation': 'Persistent dyspnea for t...,0.136109,0.893432,6.360000,0.510297,0.347374,0.510297,0.510297,34,0,28,1,3,25,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,93078,{'visit motivation': 'Anterior midline neck ma...,{'visit motivation': 'Anterior midline neck ma...,0.193582,0.898657,7.450000,0.657176,0.503265,0.638362,0.638362,25,0,23,3,2,20,73
996,38574,{'visit motivation': 'unilateral left knee pai...,"{'visit motivation': 'Left knee pain', 'admiss...",0.171704,0.901672,6.875000,0.575768,0.451841,0.563573,0.563573,16,0,19,5,6,16,62
997,65166,"{'visit motivation': 'Painful, swollen right k...",{'visit motivation': 'Painful and swollen righ...,0.210072,0.888840,7.312500,0.618466,0.490061,0.609265,0.609265,0,0,31,2,2,32,67
998,157147,{'visit motivation': 'Acute cerebral air embol...,{'visit motivation': 'Acute onset of unrespons...,0.172424,0.837329,5.000000,0.415853,0.344101,0.415853,0.415853,15,0,34,2,7,18,76


,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,gold_none_prop,pred_none_prop,missing_keys_prop
keys,,,,,,,,,,
admission/0/care center details,0.270086,0.918273,7.850427,0.735385,0.650588,0.726991,0.726991,0.048,0.153,0.012
admission/0/date,0.231097,0.919139,7.935484,0.761162,0.608599,0.754994,0.754994,0.020,0.095,0.012
admission/0/duration,0.116882,0.927714,8.314607,0.762572,0.690028,0.762572,0.762572,0.021,0.049,0.012
admission/0/reason,0.262550,0.884549,6.441617,0.552512,0.429219,0.531419,0.531419,0.038,0.069,0.012
admission/1/care center details,0.082227,0.941237,9.000000,0.795455,0.666667,0.795455,0.795455,0.001,0.003,0.010
...,...,...,...,...,...,...,...,...,...,...
treatments/5/reaction to treatment,0.000000,0.926654,8.000000,0.533333,0.461538,0.533333,0.533333,0.000,0.000,0.016
treatments/5/reason for taking,0.333333,0.809110,6.000000,0.416667,0.333333,0.416667,0.416667,0.000,0.000,0.016
treatments/5/related condition,0.000000,0.833170,5.500000,0.500000,0.000000,0.500000,0.500000,0.000,0.000,0.016
